## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Olinda,BR,-8.0089,-34.8553,78.53,78,75,12.66,broken clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,33.91,86,0,6.91,clear sky
2,2,Iracoubo,GF,5.4802,-53.2011,84.29,68,30,11.07,scattered clouds
3,3,Bethel,US,41.3712,-73.4140,68.56,88,19,3.00,few clouds
4,4,Meulaboh,ID,4.1363,96.1285,78.44,77,100,5.17,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Olinda,BR,-8.0089,-34.8553,78.53,78,75,12.66,broken clouds
2,2,Iracoubo,GF,5.4802,-53.2011,84.29,68,30,11.07,scattered clouds
4,4,Meulaboh,ID,4.1363,96.1285,78.44,77,100,5.17,overcast clouds
5,5,Kavieng,PG,-2.5744,150.7967,82.11,74,88,6.96,overcast clouds
13,13,Georgetown,MY,5.4112,100.3354,83.89,78,20,5.01,few clouds
18,18,Bengkulu,ID,-3.8004,102.2655,76.37,89,100,7.23,overcast clouds
19,19,Kankon,IN,15.0167,74.0167,76.80,85,92,5.91,overcast clouds
25,25,Atuona,PF,-9.8000,-139.0333,76.78,72,85,23.94,overcast clouds
38,38,Kendari,ID,-3.9450,122.4989,76.68,91,99,2.82,overcast clouds
39,39,Hilo,US,19.7297,-155.0900,78.28,89,100,8.05,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Weather description    241
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df =preferred_cities_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Olinda,BR,78.53,broken clouds,-8.0089,-34.8553,
2,Iracoubo,GF,84.29,scattered clouds,5.4802,-53.2011,
4,Meulaboh,ID,78.44,overcast clouds,4.1363,96.1285,
5,Kavieng,PG,82.11,overcast clouds,-2.5744,150.7967,
13,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
18,Bengkulu,ID,76.37,overcast clouds,-3.8004,102.2655,
19,Kankon,IN,76.80,overcast clouds,15.0167,74.0167,
25,Atuona,PF,76.78,overcast clouds,-9.8000,-139.0333,
38,Kendari,ID,76.68,overcast clouds,-3.9450,122.4989,
39,Hilo,US,78.28,overcast clouds,19.7297,-155.0900,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [31]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]

In [32]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Olinda,BR,78.53,broken clouds,-8.0089,-34.8553,Flat Quatro Rodas Olinda
2,Iracoubo,GF,84.29,scattered clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
4,Meulaboh,ID,78.44,overcast clouds,4.1363,96.1285,Kost Pria
5,Kavieng,PG,82.11,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
13,Georgetown,MY,83.89,few clouds,5.4112,100.3354,Cititel Penang
18,Bengkulu,ID,76.37,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
19,Kankon,IN,76.80,overcast clouds,15.0167,74.0167,Palolem Beach Resort
25,Atuona,PF,76.78,overcast clouds,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
38,Kendari,ID,76.68,overcast clouds,-3.9450,122.4989,Hotel Wixel
39,Hilo,US,78.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [33]:
len(clean_hotel_df)

223

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

In [34]:
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))